In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import numpy as np

In [2]:
import concurrent.futures

In [53]:
pd.set_option('display.notebook_repr_html', False)

In [4]:
def create_infobox(url):
    q = requests.get(url)
    webpage = bs(q.content, 'lxml')
    table_info = webpage.find_all("div", id=re.compile("receipt-.*"))
    table_info_rows = table_info[0].select('li > span')
    table_info_rows2 = table_info[0].select('li b')
    table_info_rows_text = [elem.get_text() for elem in table_info_rows]
    table_info_rows2_text = [elem.get_text(" ", strip=True) for elem in table_info_rows2]
    price_info = webpage.find('span', {'class':'priceInfo__value'}).get_text(" ", strip=True)
    price_info_m2 = webpage.find('span', {'class':'priceInfo__additional'})
    if price_info_m2 is not None:
        price_info_m2 = price_info_m2.get_text(" ", strip=True)
    else:
        price_info_m2 = 'no data'
    mapy = str(webpage.find_all('script')[-15].string)
    pattern = re.compile(r'locationParams: (.+),\n        offersId', re.DOTALL)
    tmp = pattern.search(mapy).group(1)
    data = json.loads(tmp)
    coord =  data['lokalizacja_szerokosc-geograficzna-y'], data['lokalizacja_dlugosc-geograficzna-x']
    table_info_rows_text += ['Cena', 'Cena za metr', 'Współrzędne']
    table_info_rows2_text += [price_info,price_info_m2, [coord]]
    return dict(zip(table_info_rows_text,table_info_rows2_text))

In [5]:
def list_of_urls(url):
    q = requests.get(url)
    webpage = bs(q.content, 'lxml')
    test1 = webpage.find_all('a', {'class':'teaserUnified__anchor'})
    url_list = [elem['href'] for elem in test1]
    return url_list

In [ ]:
baza_mieszkan = pd.DataFrame()
for i in range(1,75):    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for url in list_of_urls('https://gratka.pl/nieruchomosci/domy/warszawa?page=%d' %i):
            futures.append(executor.submit(create_infobox, url))
        for future in concurrent.futures.as_completed(futures):
            baza_mieszkan = baza_mieszkan.append(future.result(), ignore_index=True)         

In [22]:
baza_domow.to_csv('baza_domow.csv', index=False)

### Counting NaN in columns

In [92]:
x=pd.Series([baza_mieszkan.loc[:,column].isna().sum() for column in baza_mieszkan.columns], index=baza_mieszkan.columns, name='NaN')
x

Cena                                    0
Cena za metr                            0
Forma kuchni                         2791
Forma własności                      2988
Liczba pięter w budynku               238
Liczba pokoi                            2
Lokalizacja                             0
Numer referencyjny                      0
Piętro                                 26
Powierzchnia w m2                       0
Rok budowy                           1263
Typ zabudowy                         1522
Wspolrzedne                             0
Powierzchnia dodatkowa               2539
Miejsce parkingowe                   3750
Nazwa inwestycji                     2973
Dostępność od                        3805
Czy mieszkanie ma łazienkę?          3504
Liczba miejsc parkingowych           3488
Materiał budynku                     3556
Okna                                 4091
Stan                                 4356
Usytuowanie względem stron świata    3969
Forma / wyposażenie łazienki      

In [4]:
baza_mieszkan = pd.read_csv('baza_mieszkan.csv')
baza_domow = pd.read_csv('baza_domow.csv')

In [5]:
baza_mieszkan.drop_duplicates(inplace=True)
baza_mieszkan = baza_mieszkan.loc[:,['Cena', 'Cena za metr', 'Liczba pięter w budynku', 'Liczba pokoi', 'Lokalizacja','Piętro','Powierzchnia w m2','Wspolrzedne']]

In [6]:
baza_mieszkan['Liczba pięter w budynku']= baza_mieszkan['Liczba pięter w budynku'].str.slice(-2)
baza_mieszkan['Liczba pokoi'] = baza_mieszkan['Liczba pokoi'].str.slice(-1)

In [7]:
baza_mieszkan['Cena'] = baza_mieszkan['Cena'].str.replace('Zapytaj o cenę', '-1')
baza_mieszkan['Cena'] = baza_mieszkan['Cena'].str.replace(',', '.')
baza_mieszkan['Cena'] = baza_mieszkan['Cena'].str.replace(r'[^0-9.]+', '')

C:\Users\aszok\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
baza_mieszkan['Cena za metr'] = baza_mieszkan['Cena za metr'].str.replace(',','.')
baza_mieszkan['Cena za metr'] = baza_mieszkan['Cena za metr'].str.replace('no data', '-1 0 0')
baza_mieszkan['Cena za metr'] = baza_mieszkan['Cena za metr'].str.split(' ').str[:-2].str.join('')

In [ ]:
baza_mieszkan['Piętro']= baza_mieszkan['Piętro'].str.replace('parter', '0').str.replace(r'[^0-9]+', '')

In [10]:
baza_mieszkan['Powierzchnia w m2'] = baza_mieszkan['Powierzchnia w m2'].str.split(' ').str[:-2].str.join('').str.replace(',','.')

In [11]:
df = baza_mieszkan.Wspolrzedne

In [12]:
al = df.str.split(r'(\d+\.\d*), (\d+\.\d*)', expand=True).iloc[:,[1,2]]
al[2], al[1] = al[2].values.astype(float), al[1].values.astype(float)

In [13]:
al.columns = ['Szerokosc', 'Dlugosc']

In [14]:
baza_mieszkan.drop(['Wspolrzedne'], axis=1, inplace=True)

In [15]:
baza_mieszkan = pd.concat([baza_mieszkan,al], axis=1)

In [17]:
baza_mieszkan.Lokalizacja = baza_mieszkan.Lokalizacja.str.split(' , ').str.get(1)

In [18]:
dist_keys  = np.unique(baza_mieszkan.Lokalizacja.values)

In [19]:
dist_keys

array(['                                        Bielany/ Żoliborz, mazowieckie',
       '                                        Nowe Włochy, mazowieckie',
       'Aleksandrów', 'Bemowo', 'Białołęka', 'Białołęka Dworska',
       'Bielany', 'Brzeziny', 'Bródno', 'Błonia Wilanowskie', 'Chrzanów',
       'Czechowice', 'Czerniaków', 'Czyste', 'Falenica', 'Gocław',
       'Gocławek', 'Grochów', 'Grodzisk', 'Górny Mokotów', 'Henryków',
       'Jeziorki Południowe', 'Kawęczyn-Wygoda', 'Kobiałka', 'Ksawerów',
       'Marysin Wawerski', 'Miedzeszyn', 'Mirów', 'Międzylesie',
       'Mokotów', 'Muranów', 'Młociny', 'Niedźwiadek', 'Nowa Praga',
       'Nowe Włochy', 'Nowodwory', 'Ochota', 'Odolany', 'Opacz Wielka',
       'Powiśle', 'Powsinek', 'Praga-Południe', 'Praga-Północ', 'Pyry',
       'Radość', 'Rakowiec', 'Raków', 'Rembertów', 'Sadul',
       'Sady Żoliborskie', 'Sadyba', 'Saska Kępa', 'Sielce', 'Skorosze',
       'Stara Miłosna', 'Stara Ochota', 'Stara Praga', 'Stare Miasto',
       'Sta

In [20]:
list_district = ['Bielany', 'Włochy','Białołęka','Bemowo','Białołęka','Białołęka',
                'Bielany','Białołęka','Targówek','Wilanów','Bemowo','Ursus',
                'Mokotów','Wola','Wawer','Praga-Południe','Praga-Południe',
                'Praga-Południe','Białołęka','Mokotów','Białołęka','Ursynów',
                'Rembertów','Białołęka','Mokotów','Wawer','Wawer','Wola','Wawer',
                'Mokotów','Śródmieście','Ursynów','Ursus','Praga-Północ','Włochy',
                'Białołęka','Ochota','Wola','Włochy','Śródmieście','Wilanów','Praga-Południe',
                'Praga-Północ','Ursynów','Wawer','Ochota','Włochy','Rembertów','Wawer',
                'Żoliborz','Mokotów','Praga-Południe','Mokotów','Ursus','Wesoła', 'Ochota',
                'Praga-Północ','Śródmieście','Włochy','Mokotów','Żoliborz','Mokotów',
                'Białołęka','Ursus','Ochota','Mokotów','Mokotów','Białołęka','Targówek',
                'Wola','Ursus','Ursynów','Wawer','Mokotów','Wilanów','Wilanów','Wola',
                'Ursynów','Włochy','Targówek','Wilanów','Mazowieckie','Śródmieście','Śródmieście',
                'Białołęka','Żoliborz']

In [21]:
dict_district = dict(zip(dist_keys, list_district))

In [ ]:
baza_mieszkan.Lokalizacja = baza_mieszkan.Lokalizacja.str.replace(r'[\S\s]+',lambda m: dict_district[m.group()])

In [23]:
baza_mieszkan.loc[baza_mieszkan.Lokalizacja == 'Mazowieckie',:]

,Cena,Cena za metr,Liczba pięter w budynku,Liczba pokoi,Lokalizacja,Piętro,Powierzchnia w m2,Szerokosc,Dlugosc
13,460000,8156.03,3,2,Mazowieckie,3,56.40,52.319107,21.057799
76,1650000,10312.50,16,4,Mazowieckie,14,160,52.255887,20.987646
83,1995000,6650,3,7,Mazowieckie,0,300,52.229700,21.012200
85,492000,10784.74,4,2,Mazowieckie,4,45.62,52.208394,20.937170
100,597276,11426.75,8,2,Mazowieckie,1,52.27,52.220859,20.957236
...,...,...,...,...,...,...,...,...,...
3612,1,-1,5,3,Mazowieckie,1,57.78,52.193600,20.941200
3613,1,-1,5,4,Mazowieckie,4,72.46,52.193600,20.941200
3614,1,-1,5,3,Mazowieckie,4,63.55,52.193600,20.941200
3743,403560,7200,3,3,Mazowieckie,1,56.05,52.224392,21.227209


In [24]:
import geopandas as gpd
import os

In [25]:
shapefile = os.path.join(os.path.expanduser('~'), 'Documents', 'Python Scripts', 'dzielniceWAW','dzielniceWAW.shp')
street_map =gpd.read_file(shapefile, encoding='utf-8')

In [ ]:
gdf = gpd.GeoDataFrame(baza_mieszkan.loc[:,['Szerokosc','Dlugosc']], geometry=gpd.points_from_xy(baza_mieszkan.loc[:,['Szerokosc','Dlugosc']].Dlugosc, baza_mieszkan.loc[:,['Szerokosc','Dlugosc']].Szerokosc))
joinDF=gpd.sjoin(gdf, street_map, how='left',op="within")

In [27]:
baza_mieszkan.Lokalizacja = joinDF.nazwa_dzie

In [ ]:
baza_mieszkan = baza_mieszkan.loc[~(baza_mieszkan['Cena']> 12000000),:]

In [35]:
baza_mieszkan.to_csv('flat_database.csv', index=False)